<a href="https://colab.research.google.com/github/Kaiiser-ai/AutoCode-SBC-2024-StairChecker/blob/main/AutoCode_Stair_Pro_%E2%80%93_SBC_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet gradio

import gradio as gr
from dataclasses import dataclass
from typing import List, Dict

# =========================
# Thresholds (prototype SBC 2024)
# =========================

RISER_MIN = 150
RISER_MAX = 175
RISER_VAR_MAX = 5

TREAD_MIN = 275
TREAD_VAR_MAX = 5

MAX_RISERS_PER_FLIGHT = 16

STAIR_WIDTH_MIN = 1100
HANDRAIL_WIDTH_MIN = 1000
OCCUPANT_WIDTH_PER_PERSON = 5  # mm/person

LANDING_DEPTH_MIN_ABS = 1100
MAX_VERTICAL_RISE_BETWEEN_LANDINGS = 3600  # mm
DOOR_PROJECTION_RATIO_MAX = 0.5

HEADROOM_TREADS_MIN = 2100
HEADROOM_LANDINGS_MIN = 2300

HANDRAIL_HEIGHT_MIN = 850
HANDRAIL_HEIGHT_MAX = 950
HANDRAIL_EXTENSION_MIN = 300
HAND_CLEARANCE_MIN = 40

GUARD_HEIGHT_MIN = 1100
GUARD_OPENING_MAX = 150


# =========================
# Data model
# =========================

@dataclass
class StairData:
    stair_id: str = "Stair 1"
    building_use: str = "Residential"
    stair_type: str = "Exit stair"
    floors_served: int = 2
    accessible_route: bool = True

    riser_typ: float = 170
    riser_min: float = 170
    riser_max: float = 170
    risers_per_flight: int = 12
    tread_typ: float = 280
    tread_min: float = 280
    nosing_proj: float = 0
    open_risers: bool = False

    width_walls: float = 1200
    width_handrails: float = 1150
    occupant_load: int = 80

    top_landing_depth: float = 1200
    bottom_landing_depth: float = 1200
    max_rise_between_landings: float = 3000
    landings_level: bool = True
    door_swings_onto_landing: bool = True
    door_proj: float = 400

    headroom_treads: float = 2200
    headroom_landings: float = 2350

    has_handrail_left: bool = True
    has_handrail_right: bool = True
    handrail_height: float = 900
    handrail_continuous: bool = True
    handrail_ext_top: float = 300
    handrail_ext_bottom: float = 300
    hand_clearance: float = 45
    graspable_profile_ok: bool = True

    guard_required: bool = True
    guard_height: float = 1100
    guard_opening_max: float = 140
    large_low_opening: bool = False

    slip_resistant: bool = True
    contrast_nosings: bool = True
    tactile_warnings: bool = True


def rule_result(group: str, rule: str, passed: bool, detail: str) -> Dict:
    return {
        "group": group,
        "rule": rule,
        "pass": bool(passed),
        "detail": detail,
    }


# =========================
# Rule groups
# =========================

def check_geometry(stair: StairData) -> List[Dict]:
    r = []

    r.append(rule_result(
        "Geometry",
        f"Riser between {RISER_MIN}–{RISER_MAX} mm",
        RISER_MIN <= stair.riser_typ <= RISER_MAX,
        f"typical {stair.riser_typ} mm"
    ))

    riser_var = stair.riser_max - stair.riser_min
    r.append(rule_result(
        "Geometry",
        f"Riser variation ≤ {RISER_VAR_MAX} mm",
        riser_var <= RISER_VAR_MAX,
        f"min {stair.riser_min} mm, max {stair.riser_max} mm, variation {riser_var} mm"
    ))

    r.append(rule_result(
        "Geometry",
        f"Tread depth ≥ {TREAD_MIN} mm",
        stair.tread_typ >= TREAD_MIN,
        f"typical {stair.tread_typ} mm (min {stair.tread_min} mm)"
    ))

    tread_var = stair.tread_typ - stair.tread_min
    r.append(rule_result(
        "Geometry",
        f"Tread variation ≤ {TREAD_VAR_MAX} mm",
        tread_var <= TREAD_VAR_MAX,
        f"typical {stair.tread_typ} mm, min {stair.tread_min} mm, variation {tread_var} mm"
    ))

    r.append(rule_result(
        "Geometry",
        f"Risers per flight ≤ {MAX_RISERS_PER_FLIGHT}",
        stair.risers_per_flight <= MAX_RISERS_PER_FLIGHT,
        f"{stair.risers_per_flight} risers"
    ))

    return r


def check_width_capacity(stair: StairData) -> List[Dict]:
    r = []

    r.append(rule_result(
        "Width & Capacity",
        f"Clear width between walls ≥ {STAIR_WIDTH_MIN} mm",
        stair.width_walls >= STAIR_WIDTH_MIN,
        f"{stair.width_walls} mm"
    ))

    r.append(rule_result(
        "Width & Capacity",
        f"Clear width between handrails ≥ {HANDRAIL_WIDTH_MIN} mm",
        stair.width_handrails >= HANDRAIL_WIDTH_MIN,
        f"{stair.width_handrails} mm"
    ))

    required_width = max(STAIR_WIDTH_MIN, stair.occupant_load * OCCUPANT_WIDTH_PER_PERSON)
    r.append(rule_result(
        "Width & Capacity",
        "Width between handrails ≥ required width from load (max(1100, 5 mm × load))",
        stair.width_handrails >= required_width,
        f"occupant load {stair.occupant_load}, required {required_width} mm, provided {stair.width_handrails} mm"
    ))

    return r


def check_landings_doors(stair: StairData) -> List[Dict]:
    r = []

    req_depth_top = max(LANDING_DEPTH_MIN_ABS, stair.width_walls)
    req_depth_bottom = max(LANDING_DEPTH_MIN_ABS, stair.width_walls)

    r.append(rule_result(
        "Landings & Doors",
        "Top landing depth ≥ max(1100 mm, stair width)",
        stair.top_landing_depth >= req_depth_top,
        f"top landing {stair.top_landing_depth} mm, required {req_depth_top} mm"
    ))

    r.append(rule_result(
        "Landings & Doors",
        "Bottom landing depth ≥ max(1100 mm, stair width)",
        stair.bottom_landing_depth >= req_depth_bottom,
        f"bottom landing {stair.bottom_landing_depth} mm, required {req_depth_bottom} mm"
    ))

    r.append(rule_result(
        "Landings & Doors",
        f"Vertical rise between landings ≤ {MAX_VERTICAL_RISE_BETWEEN_LANDINGS} mm",
        stair.max_rise_between_landings <= MAX_VERTICAL_RISE_BETWEEN_LANDINGS,
        f"{stair.max_rise_between_landings} mm"
    ))

    if stair.door_swings_onto_landing:
        allowed_proj = stair.bottom_landing_depth * DOOR_PROJECTION_RATIO_MAX
        r.append(rule_result(
            "Landings & Doors",
            "Door projection ≤ 1/2 landing depth when swinging onto landing",
            stair.door_proj <= allowed_proj,
            f"door projection {stair.door_proj} mm, half landing {allowed_proj} mm"
        ))
    else:
        r.append(rule_result(
            "Landings & Doors",
            "Door does not swing onto landing",
            True,
            "no encroachment"
        ))

    r.append(rule_result(
        "Landings & Doors",
        "Landings level / acceptable slope",
        stair.landings_level,
        f"landings_level = {stair.landings_level}"
    ))

    return r


def check_headroom(stair: StairData) -> List[Dict]:
    r = []
    r.append(rule_result(
        "Headroom",
        f"Headroom above treads ≥ {HEADROOM_TREADS_MIN} mm",
        stair.headroom_treads >= HEADROOM_TREADS_MIN,
        f"{stair.headroom_treads} mm"
    ))
    r.append(rule_result(
        "Headroom",
        f"Headroom above landings ≥ {HEADROOM_LANDINGS_MIN} mm",
        stair.headroom_landings >= HEADROOM_LANDINGS_MIN,
        f"{stair.headroom_landings} mm"
    ))
    return r


def check_handrails(stair: StairData) -> List[Dict]:
    r = []

    exit_stair = (stair.stair_type == "Exit stair")
    if exit_stair:
        r.append(rule_result(
            "Handrails",
            "Exit stair: handrails on both sides",
            stair.has_handrail_left and stair.has_handrail_right,
            f"left: {stair.has_handrail_left}, right: {stair.has_handrail_right}"
        ))
    else:
        r.append(rule_result(
            "Handrails",
            "Non-exit stair: at least one handrail",
            stair.has_handrail_left or stair.has_handrail_right,
            f"left: {stair.has_handrail_left}, right: {stair.has_handrail_right}"
        ))

    r.append(rule_result(
        "Handrails",
        f"Handrail height {HANDRAIL_HEIGHT_MIN}–{HANDRAIL_HEIGHT_MAX} mm",
        HANDRAIL_HEIGHT_MIN <= stair.handrail_height <= HANDRAIL_HEIGHT_MAX,
        f"{stair.handrail_height} mm"
    ))

    r.append(rule_result(
        "Handrails",
        f"Handrail top extension ≥ {HANDRAIL_EXTENSION_MIN} mm",
        stair.handrail_ext_top >= HANDRAIL_EXTENSION_MIN,
        f"{stair.handrail_ext_top} mm"
    ))

    r.append(rule_result(
        "Handrails",
        f"Handrail bottom extension ≥ {HANDRAIL_EXTENSION_MIN} mm",
        stair.handrail_ext_bottom >= HANDRAIL_EXTENSION_MIN,
        f"{stair.handrail_ext_bottom} mm"
    ))

    r.append(rule_result(
        "Handrails",
        f"Hand clearance from wall ≥ {HAND_CLEARANCE_MIN} mm",
        stair.hand_clearance >= HAND_CLEARANCE_MIN,
        f"{stair.hand_clearance} mm"
    ))

    r.append(rule_result(
        "Handrails",
        "Handrails continuous between landings",
        stair.handrail_continuous,
        f"continuous = {stair.handrail_continuous}"
    ))

    r.append(rule_result(
        "Handrails",
        "Graspable profile compliant (manual check)",
        stair.graspable_profile_ok,
        f"graspable_profile_ok = {stair.graspable_profile_ok}"
    ))

    return r


def check_guards(stair: StairData) -> List[Dict]:
    r = []
    if stair.guard_required:
        r.append(rule_result(
            "Guards",
            f"Guard height ≥ {GUARD_HEIGHT_MIN} mm",
            stair.guard_height >= GUARD_HEIGHT_MIN,
            f"{stair.guard_height} mm"
        ))
        r.append(rule_result(
            "Guards",
            f"Guard opening ≤ {GUARD_OPENING_MAX} mm",
            stair.guard_opening_max <= GUARD_OPENING_MAX,
            f"{stair.guard_opening_max} mm"
        ))
        r.append(rule_result(
            "Guards",
            "No large low-level openings > 100 mm",
            not stair.large_low_opening,
            f"large_low_opening = {stair.large_low_opening}"
        ))
    else:
        r.append(rule_result(
            "Guards",
            "Guard not required",
            True,
            "guard_required = False"
        ))
    return r


def check_safety(stair: StairData) -> List[Dict]:
    r = []
    r.append(rule_result(
        "Safety / Other",
        "Slip-resistant surface",
        stair.slip_resistant,
        f"slip_resistant = {stair.slip_resistant}"
    ))
    r.append(rule_result(
        "Safety / Other",
        "Contrast on nosings (where required)",
        stair.contrast_nosings,
        f"contrast_nosings = {stair.contrast_nosings}"
    ))
    r.append(rule_result(
        "Safety / Other",
        "Tactile warnings at top/bottom (where required)",
        stair.tactile_warnings,
        f"tactile_warnings = {stair.tactile_warnings}"
    ))
    return r


def check_all(stair: StairData) -> List[Dict]:
    results: List[Dict] = []
    results += check_geometry(stair)
    results += check_width_capacity(stair)
    results += check_landings_doors(stair)
    results += check_headroom(stair)
    results += check_handrails(stair)
    results += check_guards(stair)
    results += check_safety(stair)
    return results


def summarize_markdown(results: List[Dict]) -> str:
    overall_pass = all(r["pass"] for r in results)
    md = f"## Overall: **{'COMPLIANT ✅' if overall_pass else 'NON-COMPLIANT ❌'}**\n\n"
    md += "### SBC 2024 Staircase Rule Checks\n\n"

    groups: Dict[str, List[Dict]] = {}
    for r in results:
        groups.setdefault(r["group"], []).append(r)

    for group, items in groups.items():
        md += f"#### {group}\n"
        for item in items:
            status = "🟢 PASS" if item["pass"] else "🔴 FAIL"
            md += f"- **{status}** — {item['rule']} (_{item['detail']}_) \n"
        md += "\n"

    return md


# =========================
# Gradio UI
# =========================

def run_checker(
    stair_id, building_use, stair_type, floors_served, accessible_route,
    riser_typ, riser_min, riser_max, risers_per_flight, tread_typ, tread_min, nosing_proj, open_risers,
    width_walls, width_handrails, occupant_load,
    top_landing_depth, bottom_landing_depth, max_rise_between_landings, landings_level,
    door_swings_onto_landing, door_proj,
    headroom_treads, headroom_landings,
    has_handrail_left, has_handrail_right, handrail_height, handrail_continuous,
    handrail_ext_top, handrail_ext_bottom, hand_clearance, graspable_profile_ok,
    guard_required, guard_height, guard_opening_max, large_low_opening,
    slip_resistant, contrast_nosings, tactile_warnings
):
    stair = StairData(
        stair_id=stair_id,
        building_use=building_use,
        stair_type=stair_type,
        floors_served=int(floors_served),
        accessible_route=accessible_route,

        riser_typ=float(riser_typ),
        riser_min=float(riser_min),
        riser_max=float(riser_max),
        risers_per_flight=int(risers_per_flight),
        tread_typ=float(tread_typ),
        tread_min=float(tread_min),
        nosing_proj=float(nosing_proj),
        open_risers=open_risers,

        width_walls=float(width_walls),
        width_handrails=float(width_handrails),
        occupant_load=int(occupant_load),

        top_landing_depth=float(top_landing_depth),
        bottom_landing_depth=float(bottom_landing_depth),
        max_rise_between_landings=float(max_rise_between_landings),
        landings_level=landings_level,
        door_swings_onto_landing=door_swings_onto_landing,
        door_proj=float(door_proj),

        headroom_treads=float(headroom_treads),
        headroom_landings=float(headroom_landings),

        has_handrail_left=has_handrail_left,
        has_handrail_right=has_handrail_right,
        handrail_height=float(handrail_height),
        handrail_continuous=handrail_continuous,
        handrail_ext_top=float(handrail_ext_top),
        handrail_ext_bottom=float(handrail_ext_bottom),
        hand_clearance=float(hand_clearance),
        graspable_profile_ok=graspable_profile_ok,

        guard_required=guard_required,
        guard_height=float(guard_height),
        guard_opening_max=float(guard_opening_max),
        large_low_opening=large_low_opening,

        slip_resistant=slip_resistant,
        contrast_nosings=contrast_nosings,
        tactile_warnings=tactile_warnings
    )

    results = check_all(stair)
    md = summarize_markdown(results)
    return md


with gr.Blocks() as demo:
    gr.Markdown("# 🏛️ AutoCode Stair Pro – SBC 2024")
    gr.Markdown("Full staircase compliance checker (rule-based, no API).")

    with gr.Accordion("Basic Info", open=True):
        with gr.Row():
            stair_id = gr.Textbox(label="Stair ID / Name", value="Stair 1")
            building_use = gr.Dropdown(
                label="Building use",
                choices=["Residential", "Office", "Assembly", "Healthcare", "Other"],
                value="Residential"
            )
            stair_type = gr.Dropdown(
                label="Stair type",
                choices=["Exit stair", "Internal circulation", "Private stair"],
                value="Exit stair"
            )
        with gr.Row():
            floors_served = gr.Number(label="Floors served", value=2, precision=0)
            accessible_route = gr.Checkbox(label="Accessible route", value=True)

    with gr.Accordion("Geometry", open=False):
        with gr.Row():
            riser_typ = gr.Number(label="Typical riser height (mm)", value=170)
            riser_min = gr.Number(label="Minimum riser height (mm)", value=170)
            riser_max = gr.Number(label="Maximum riser height (mm)", value=170)
            risers_per_flight = gr.Number(label="Risers per flight", value=12, precision=0)
        with gr.Row():
            tread_typ = gr.Number(label="Typical tread depth (mm)", value=280)
            tread_min = gr.Number(label="Minimum tread depth (mm)", value=280)
            nosing_proj = gr.Number(label="Nosing projection (mm)", value=0)
            open_risers = gr.Checkbox(label="Open risers", value=False)

    with gr.Accordion("Width & Capacity", open=False):
        with gr.Row():
            width_walls = gr.Number(label="Clear width between walls (mm)", value=1200)
            width_handrails = gr.Number(label="Width between handrails (mm)", value=1150)
            occupant_load = gr.Number(label="Occupant load served (persons)", value=80, precision=0)

    with gr.Accordion("Landings & Doors", open=False):
        with gr.Row():
            top_landing_depth = gr.Number(label="Top landing depth (mm)", value=1200)
            bottom_landing_depth = gr.Number(label="Bottom landing depth (mm)", value=1200)
            max_rise_between_landings = gr.Number(label="Max vertical rise between landings (mm)", value=3000)
        with gr.Row():
            landings_level = gr.Checkbox(label="Landings level / acceptable slope", value=True)
            door_swings_onto_landing = gr.Checkbox(label="Door swings onto landing", value=True)
            door_proj = gr.Number(label="Door leaf projection (mm)", value=400)

    with gr.Accordion("Headroom", open=False):
        with gr.Row():
            headroom_treads = gr.Number(label="Headroom above treads (mm)", value=2200)
            headroom_landings = gr.Number(label="Headroom above landings (mm)", value=2350)

    with gr.Accordion("Handrails", open=False):
        with gr.Row():
            has_handrail_left = gr.Checkbox(label="Handrail on left", value=True)
            has_handrail_right = gr.Checkbox(label="Handrail on right", value=True)
            handrail_height = gr.Number(label="Handrail height (mm)", value=900)
            handrail_continuous = gr.Checkbox(label="Handrails continuous", value=True)
        with gr.Row():
            handrail_ext_top = gr.Number(label="Top extension length (mm)", value=300)
            handrail_ext_bottom = gr.Number(label="Bottom extension length (mm)", value=300)
            hand_clearance = gr.Number(label="Hand clearance from wall (mm)", value=45)
            graspable_profile_ok = gr.Checkbox(label="Graspable profile compliant", value=True)

    with gr.Accordion("Guards", open=False):
        with gr.Row():
            guard_required = gr.Checkbox(label="Guard required", value=True)
            guard_height = gr.Number(label="Guard height (mm)", value=1100)
            guard_opening_max = gr.Number(label="Max guard opening (mm)", value=140)
            large_low_opening = gr.Checkbox(label="Any large low-level opening > 100 mm", value=False)

    with gr.Accordion("Safety / Other", open=False):
        with gr.Row():
            slip_resistant = gr.Checkbox(label="Slip-resistant surface", value=True)
            contrast_nosings = gr.Checkbox(label="Contrast on nosings", value=True)
            tactile_warnings = gr.Checkbox(label="Tactile warnings at top/bottom", value=True)

    run_btn = gr.Button("Run SBC 2024 Check")
    output_md = gr.Markdown()

    run_btn.click(
        run_checker,
        inputs=[
            stair_id, building_use, stair_type, floors_served, accessible_route,
            riser_typ, riser_min, riser_max, risers_per_flight, tread_typ, tread_min, nosing_proj, open_risers,
            width_walls, width_handrails, occupant_load,
            top_landing_depth, bottom_landing_depth, max_rise_between_landings, landings_level,
            door_swings_onto_landing, door_proj,
            headroom_treads, headroom_landings,
            has_handrail_left, has_handrail_right, handrail_height, handrail_continuous,
            handrail_ext_top, handrail_ext_bottom, hand_clearance, graspable_profile_ok,
            guard_required, guard_height, guard_opening_max, large_low_opening,
            slip_resistant, contrast_nosings, tactile_warnings
        ],
        outputs=output_md
    )

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fd112dd2ea8d0480a7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
